In [1]:
import os
from os.path import expanduser
import socket
import sys
import glob

# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemOLLIE = 'ollie0'
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "pyfesom")
elif running_system[0:1] == "o" or running_system[0:1] == "b" or running_system == systemHLRN or running_system == systemOLLIE:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"pyfesom")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules") # add custom Vibe 's modules
#sys.path.append("pyfesom") # add standard 's modules
#########################################

computer =  ollie0
working directory set to /home/ollie/loziel/pyfesom


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
import pyfesom as pf
#from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import cmocean as cmaps
from matplotlib.colors import ListedColormap
import colormaps as cmaps

no cmocean


In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = ListedColormap(cmaps.viridis.colors)
# choose simulation
runid='Arc12'
# choose depth
get3d = False
# param
var = 'DSi'
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/'
#resultpath = '/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/'
savepath    = '/work/ollie/loziel/figures/riverchemistry/'
meshpath    = '/work/ollie/loziel/mesh/meshArc4.5_notrotated/'


# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /work/ollie/loziel/figures/riverchemistry/ already existing


In [4]:
mesh = pf.load_mesh(meshpath, abg = [0., 0., 0.], usepickle=False,get3d=get3d)

Load 2d part of the grid in 29 second(s)


In [5]:
mesh


FESOM mesh:
path                  = /work/ollie/loziel/mesh/meshArc4.5_notrotated
alpha, beta, gamma    = 0.0, 0.0, 0.0
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 0

        

In [6]:
elem2=mesh.elem[mesh.no_cyclic_elem,:]

In [7]:
months =np.linspace(0,11,12).astype(int)

In [8]:
for mo in months:
    # loop over years
    ncfile = resultpath+'River.'+var+'.nc'
    print ncfile
    f      = Dataset(ncfile, 'r')
    runoff    = f.variables[var][mo,0:mesh.n2d]
    ##################
    data2 = runoff
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2)
    print 'Max and min: ',np.max(data2),np.min(data2)    
        
    contours = [0, 300, 10]
    contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
    #contours=np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
    #fig.suptitle('Density (kg m$^{-3}$)')
    fig.subplots_adjust(wspace=0.02,hspace=0.02)
    elem2=mesh.elem[mesh.no_cyclic_elem,:]
    d = data2[elem2].mean(axis=1)
    k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
    elem2=elem2[k,:]
    print 'ftriplot, number of dummy points:', len(d)-len(k)		
    map = Basemap(projection='robin',lon_0=0) 
    x, y = map(mesh.x2, mesh.y2)
    map.drawmapboundary(fill_color='0.9')
    map.drawcoastlines()
    mlabels=[False,False,False,False]
    plabels=[True,True,True,True]		    
    map.drawparallels(np.arange(-90,90,30),labels=plabels) 
    map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels)
    eps=(contours.max()-contours.min())/100.
    data2[data2<=contours.min()]=contours.min()+eps
    data2[data2>=contours.max()]=contours.max()-eps
    im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))

    #cbar_ax = fig.add_axes([0.14, 0.2, 0.75, 0.02])
    cb=map.colorbar(im,"bottom", size="5%", pad="2%")#, format='%.0e'
    cb.ax.tick_params(labelsize='large')
    cb.set_label(label=var+' [mmol Si day$^{-1}$]', size='large')
    plt.title('River '+var+' runoff - '+str(mo+1))
    #plt.show() 
    if mo < 9:
        plt.savefig(savepath+var+'_0'+str(mo+1)+'_Global.png', dpi = 200, bbox_inches='tight')
    else:
        plt.savefig(savepath+var+'_'+str(mo+1)+'_Global.png', dpi = 200, bbox_inches='tight')
    plt.close()

/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.23130983894
Max and min:  3515.39512165 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.31564236606
Max and min:  4077.16969912 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.48666489017
Max and min:  4654.64777042 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.18885462806
Max and min:  5117.88920178 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0


In [9]:
for mo in months:
    # loop over years
    ncfile = resultpath+'River.'+var+'.nc'
    print ncfile
    f      = Dataset(ncfile, 'r')
    runoff    = f.variables[var][mo,0:mesh.n2d]
    ##################
    data2 = runoff
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2)
    print 'Max and min: ',np.max(data2),np.min(data2)    
        
    contours = [0, 300, 10]
    contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
    #contours=np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
    fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
    #fig.suptitle('Density (kg m$^{-3}$)')
    fig.subplots_adjust(wspace=0.02,hspace=0.02)
    elem2=mesh.elem[mesh.no_cyclic_elem,:]
    d = data2[elem2].mean(axis=1)
    k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
    elem2=elem2[k,:]
    print 'ftriplot, number of dummy points:', len(d)-len(k)		
    map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
    x, y = map(mesh.x2, mesh.y2)
    map.drawmapboundary(fill_color='0.9')
    map.fillcontinents(color='.5',lake_color='.7')
    map.drawcoastlines()
    plabels=[False,False,False,False]
    mlabels=[False,False,False,False]    
    map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
    map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
    eps=(contours.max()-contours.min())/100.
    data2[data2<=contours.min()]=contours.min()+eps
    data2[data2>=contours.max()]=contours.max()-eps
    im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))

    #cbar_ax = fig.add_axes([0.14, 0.2, 0.75, 0.02])
    cb=map.colorbar(im,"bottom", size="5%", pad="2%")#, format='%.0e'
    cb.ax.tick_params(labelsize='large')
    cb.set_label(label=var+' [mmol Si day$^{-1}$]', size='large')
    plt.title('River '+var+' runoff - '+str(mo+1))
    #plt.show() 
    if mo < 9:
        plt.savefig(savepath+var+'_0'+str(mo+1)+'_Arctic.png', dpi = 200, bbox_inches='tight')
    else:
        plt.savefig(savepath+var+'_'+str(mo+1)+'_Arctic.png', dpi = 200, bbox_inches='tight')
    plt.close()

/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.23130983894
Max and min:  3515.39512165 0.0
ftriplot, number of dummy points: 0


/global/AWIsoft/intel/2018/intelpython2/lib/python2.7/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()


/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.31564236606
Max and min:  4077.16969912 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.48666489017
Max and min:  4654.64777042 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.18885462806
Max and min:  5117.88920178 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  2.52313715988
Max and min:  5254.74748439 0.0
ftriplot, number of dummy points: 0
/work/ollie/loziel/Arc4.5_REcoMforcing_notrotated/River.DSi.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
